# Dependencies

In [1]:
import igraph as ig
import random
import subprocess
from collections import defaultdict
import numpy as np
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import time
import pickle
from collections import Counter
import networkx as nx

# Common part

## Uniqueness

In [2]:
# Compuate equivalent class.
def ig_to_nauty(g):
    graph = "n " + str(g.vcount()) + "\n"
    graph += "g\n"
    for n in g.vs.indices:
        graph += str(n) + ":"
        for m in g.neighbors(n):
            graph += " " + str(m)
        graph += ";\n"
    
    return graph

def computeEQ(G, criteria='dk'):
    '''
    Input: orignal graph
    Output: a dict of equivalent class and the list of nodes of the eqvialent class. i.e. {'class id': [nodes]}
    '''
    P = defaultdict(list)
    if criteria == 'dk':
        ng = ig_to_nauty(G)
        cmd = ng + "d " + str(1) + "\na\n"
        cmd += ".\n"
        r = subprocess.Popen(['./dkAnonymity/src/menu'], 
                            stdin=subprocess.PIPE, 
                            stdout=subprocess.PIPE, 
                            stderr=subprocess.PIPE, 
                            shell=True)
        res = r.communicate(cmd.encode())[0].decode()
        str_p = res.split("\n")
        str_p = str_p[str_p.index("node, class_id")+1: str_p.index("End eq map")]

        for i in str_p:
            p = i.split(", ")
            P[p[1]].append(int(p[0]))
    
    elif criteria == 'nm':
        for node in G.vs.indices:
            neighborhood = G.neighbors(node)
            n = len(neighborhood) + 1
            neighborhood.append(node)
            sg = G.subgraph(neighborhood)
            m = sg.ecount()
            P[(n, m)].append(node)

    return P

In [80]:
# Compute the uniqueness of the network.
def uniqueness(G, criteria):
    '''
    Input: original graph
    Output: a number of uniquendess, and a list of unique nodes
    '''
    u = 0
    n = G.vcount()
    p = computeEQ(G, criteria)
    unique_nodes = set()
    for cl in p:
        if len(p[cl]) < 2:
            u += 1/n
            unique_nodes.update(p[cl])
    
    return u, unique_nodes

## Initialization

In [379]:
G_origin = nx.Graph()
# read data

# copenet-sms
# with open('sms.csv', newline='') as f:
#     lines = csv.reader(f, delimiter=',')
#     for line in lines:
#         G_origin.add_edge(line[1], line[2])

# copenet-fb
# with open('fb_friends.csv', newline='') as f:
#     lines = csv.reader(f, delimiter=',')
#     for line in lines:
#         G_origin.add_edge(line[0], line[1])

# # CollegeMsg
# with open('CollegeMsg.txt') as f:
#         for line in f:
#             edge = line.strip().split(' ')
#             G_origin.add_edge(edge[0], edge[1])

# ca-grqc        
with open('CA-GrQc.txt') as f:
        for line in f:
            edge = line.strip().split('\t')
            G_origin.add_edge(edge[0], edge[1])

# ego-fb
# with open('facebook_combined.txt') as f:
#         for line in f:
#             edge = line.strip().split(' ')
#             G_origin.add_edge(edge[0], edge[1])

G_origin = ig.Graph.from_networkx(G_origin)
U, unodes_origin = uniqueness(G_origin, 'dk')
# print(unodes_origin)

# info of empirical network
ig.summary(G_origin)
print("ACC:", G_origin.transitivity_avglocal_undirected(mode="zero"), "Transitivity:", G_origin.transitivity_undirected())
print("U_nm:", U)

IGRAPH U--- 5242 14496 -- 
+ attr: _nx_name (v)
ACC: 0.529635811052136 Transitivity: 0.6298424741263426
U_nm: 0.13506295307134755


In [380]:
# initialize
STEP = 25
CALCULATE_TIME = G_origin.ecount() // STEP

x = list(range(STEP, (CALCULATE_TIME+1)*STEP, STEP))
u_of_10 = np.full((10, CALCULATE_TIME+1), U)

# Deletion

## Random Delete

In [137]:
# Random deletion
def randomDelete(G, num):
    edges = random.sample(G.es.indices, num)
    G.delete_edges(edges)

    return G

In [138]:
def random1Round(G, U, STEP, CALCULATE_TIME, criteria):
    u_random = [U]

    t0 = time.time()
    for i in range(CALCULATE_TIME):
        # random delete
        G = randomDelete(G, STEP)
        new_u_random, _ = uniqueness(G, criteria)
        u_random.append(new_u_random)
    #     print("u_random:", new_u_random)
    t1 = time.time()
    
    return u_random, t1-t0

In [24]:
G_random = G_origin.copy()

u_random, t_random = random1Round(G_random, U, STEP, CALCULATE_TIME, 'nm')
print('Time of random delete: {:.3f}s'.format(t_random))

with open('./imtermediate results/RandomNM.txt', 'wb') as f:
    pickle.dump(u_random, f)

Time of random delete: 8.931s


## Max-degree

In [6]:
# Find all edges between unique nodes.
def getEdgeListUnique(G, node_list):
    '''
    Input: original graph, and list of unique nodes
    Output: list of edges between unique nodes
    '''
    edges = G.get_edgelist()
    res = []
    for edge in edges:
        if (edge[0] in node_list) and (edge[1] in node_list):
            res.append(edge)
    
    return res

In [149]:
# Delete edges based on the maximum degrees of the connected nodes.
def deleteMax(G, num, edge_list):
    '''
    Input: 
        G: original graph, 
        num: number of edges need to be deleted, 
        edges_list: list edges that only touch the unique nodes.
    Output: modified graph
    '''
    p = []
    edges = []
    if num < len(edge_list):    # if the edges between unique nodes are enough
        edge_candidates = edge_list.copy()
    else:
        edge_candidates = G.get_edgelist()
    
    for j in edge_candidates:
        nodes_deg = 1/max(G.degree(j[0]), G.degree(j[1]))
        p.append(nodes_deg)
    for _ in range(num):
        edge = random.choices(edge_candidates, weights=tuple(p))[0]
        index = edge_candidates.index(edge)
        edges.append(G.get_eid(edge[0], edge[1]))
        p[index] = 0
    
    G.delete_edges(edges)

    return G

In [150]:
def max1Round(G, U, edge_list, STEP, CALCULATE_TIME, criteria):
    u_max = [U]

    t0 = time.time()
    for i in range(CALCULATE_TIME):
        # delete-max
        G = deleteMax(G, STEP, edge_list)
        # update info of delete-max
        new_u_max, unodes_max = uniqueness(G, criteria)
        u_max.append(new_u_max)
        edge_list = getEdgeListUnique(G, unodes_max)
    #     print("u_max:", new_u_max)
    t1 = time.time()
    
    return u_max, t1-t0

In [32]:
G_max = G_origin.copy()
edge_list = getEdgeListUnique(G_origin, unodes_origin)

u_max, t_max = max1Round(G_max, U, edge_list, STEP, CALCULATE_TIME, 'nm')
print('Time of delete-max: {:.3f}s'.format(t_max))

with open('./imtermediate results/MaxNM.txt', 'wb') as f:
    pickle.dump(u_max, f)

Time of delete-max: 11.823s


## U/A

In [7]:
# Compute the U and A of all edges.
def edgeUA(G, unodes):
    '''
    Input: original graph, and list of unique nodes
    Output: U, A, and directly connected unique nodes
    '''
    edges = G.get_edgelist()
    res = {}
    for u, v in edges:
        f = []
        node_list = [u, v]
        neighbors_u = set(G.neighbors(u))
        neighbors_v = set(G.neighbors(v))
        node_list += list(neighbors_u.intersection(neighbors_v))
        for node in node_list:
            if node in unodes:
                f.append(1)
            else:
                f.append(0)
        res[u,v] = (sum(f), len(f)-sum(f), f[0]+f[1])

    return res

In [152]:
# Delete edges based on U/A
def deleteUA(G, num, ua):
    '''
    Input:
        G: original graph
        num: number of edges need to be deleted
        ua: U, A, number of directly connected unique nodes
    Output: modified graph
    '''
    p = []
    edges = []
    edge_candidates = G.get_edgelist()
    for j in edge_candidates:
        p.append((ua[j][0]+0.01)/(ua[j][1]+0.01))
    for _ in range(num):
        edge = random.choices(edge_candidates, weights=tuple(p))[0]
        index = edge_candidates.index(edge)
        edges.append(G.get_eid(edge[0], edge[1]))
        p[index] = 0

    G.delete_edges(edges)

    return G

In [153]:
def UA1Round(G, U, edge_vec, STEP, CALCULATE_TIME, criteria):
    u_ua = [U]
    
    t0 = time.time()
    for i in range(CALCULATE_TIME):
        # delete-U/A
        G = deleteUA(G, STEP, edge_vec)
        new_u_ua, unodes_ua = uniqueness(G, criteria)
        u_ua.append(new_u_ua)
        edge_vec = edgeUA(G, unodes_ua)
    #     print("u_ua:", new_u_ua)
    t1 = time.time()
    
    return u_ua, t1-t0

In [43]:
G_ua = G_origin.copy()
edge_vec = edgeUA(G_origin, unodes_origin)

u_ua, t_ua = UA1Round(G_ua, U, edge_vec, STEP, CALCULATE_TIME, 'dk')
print('Time of delete-U/A: {:.3f}s'.format(t_ua))

with open('./imtermediate results/UANM.txt', 'wb') as f:
    pickle.dump(u_ua, f)

Time of delete-U/A: 94.756s


## Logistics Regression

In [8]:
# Transfer effect of uniqueness to label.
def value2label(x):
    if x>=0:
        return 0
    else:
        return 1

In [9]:
# Delete edges based on Logistic Regression Classification.
def LRDelete(G, num, model, unodes):
    '''
    Input:
        G: original graph
        num: the number of edges need to be deleted
        model: the trained classifier
        unodes: list of unique nodes
    Output: modified graph
    '''
    edges = G.get_edgelist()
    edge_vector = edgeUA(G, unodes)
    n = G.vcount()
    m = G.ecount()
    df = pd.DataFrame(columns=['N_nodes', 'N_edges', 'max', 'min', 'U', 'A', 'direct_connected_nodes', 'triangles'])
    for e in edges:
        df.loc[G.get_eid(e[0], e[1]), :] = [n, m, max(G.degree(e[0]), G.degree(e[1])), min(G.degree(e[0]), G.degree(e[1])), 
                                edge_vector[e][0], edge_vector[e][1], edge_vector[e][2], edge_vector[e][0]+edge_vector[e][1]-2]
    p = model.predict_proba(df)
    p = p[:,1] - p[:,0]
    index = np.argsort(p)[-num:]
    G.delete_edges([G.get_eid(edges[i][0], edges[i][1]) for i in index])
    
    return G

In [51]:
# train LR model
train = pd.read_csv('training_data_2.csv', index_col=0)
train['eff'] = train['eff'].apply(lambda x: value2label(x))
classifier = LogisticRegression(penalty='l1', solver='liblinear')
classifier.fit(train.iloc[:, :8], train.iloc[:, 8])
print(classifier.score(train.iloc[:, :8], train.iloc[:, 8]))
print("Coef:", classifier.coef_[0])

0.9015755415924224
Coef: [ 7.88699707e-04 -3.15945549e-04 -3.43509936e-02 -1.79220651e-01
  0.00000000e+00 -2.93035951e-02  1.86289269e+00  1.30494646e-01]


In [87]:
def LR1Round(G, U, unodes_origin, STEP, CALCULATE_TIME, criteria):
    u_lr = [U]
    unodes = unodes_origin.copy()
    t_u = t_d = 0

    for i in range(CALCULATE_TIME):
        # delete-LR
        t0 = time.time()
        G = LRDelete(G, STEP, classifier, unodes)
        t1 = time.time()
        # update info of delete-ua
        new_u_lr, unodes = uniqueness(G, criteria)
        t2 = time.time()
        u_lr.append(new_u_lr)
        t_d += t1-t0
        t_u += t2-t1
    #     print("u_lr:", new_u_lr)
    
    return u_lr, t_d, t_u

In [20]:
G_lr = G_origin.copy()

u_lr, t_lr = LR1Round(G_lr, U, STEP, CALCULATE_TIME, 'nm')
print('Time of LR: {:.3f}s'.format(t_lr))

with open('./imtermediate results/LRNM.txt', 'wb') as f:
    pickle.dump(u_lr, f)

Time of LR: 707.717s


## Greedy

In [110]:
def getNodeVector(G):
    res = {}
    for node in G.vs.indices:
        neighborhood = G.neighbors(node)
        neighborhood.append(node)
        sg = G.subgraph(neighborhood)    # ego network
        res[node] = [len(neighborhood), sg.ecount()]
    
    return res

def delEdgeEff(G, edge, node_vec):
    eff = 0
    common_neighbors = set(G.neighbors(edge[0])).intersection(set(G.neighbors(edge[1])))
    P = Counter([tuple(node_vec[i]) for i in node_vec])
    
    for node in common_neighbors:    # indirectly related nodes
        n, m = tuple(node_vec[node])
        P[(n,m)] -= 1
        if P[(n,m)] == 1: eff -= 1
        elif P[(n,m)] == 0: eff += 1
        m -= 1
        if (n, m) not in P: eff -= 1
        elif P[(n,m)] == 1: eff += 1
        P[(n,m)] += 1

    # directly related nodes
    n, m = tuple(node_vec[edge[0]])
    P[(n,m)] -= 1
    if P[(n,m)] == 1: eff -= 1
    elif P[(n,m)] == 0: eff += 1

    n -= 1
    m -= len(common_neighbors)+1
    if (n, m) not in P: eff -= 1
    elif P[(n,m)] == 1: eff += 1
    P[(n,m)] += 1
    
    n, m = tuple(node_vec[edge[1]])
    P[tuple(node_vec[edge[1]])] -= 1
    if P[(n,m)] == 1: eff -= 1
    elif P[(n,m)] == 0: eff += 1
    
    n -= 1
    m -= len(common_neighbors)+1
    if (n, m) not in P: eff -= 1
    elif P[(n,m)] == 1: eff += 1
    P[(n,m)] += 1

    return eff

In [111]:
def greedyDelete(G, num, node_vec=None):
    edges = G.get_edgelist()
    if not node_vec:
        node_vec = getNodeVector(G)
    values = []
    
    for edge in edges:
        values.append(delEdgeEff(G, edge, node_vec))
    sorted_edges = sorted([i for i in enumerate(values)], key=lambda x: x[1])[-num:]
    G.delete_edges([G.get_eid(edges[i[0]][0], edges[i[0]][1]) for i in sorted_edges])
    
    return G

In [91]:
def greedy1Round(G, U, STEP, CALCULATE_TIME, criteria):
    u = [U]
    t_u = t_d = 0
    for i in range(CALCULATE_TIME):
        t0 = time.time()
        G = greedyDelete(G, STEP)
        t1 = time.time()
        new_u, _ = uniqueness(G, criteria)
        t2 = time.time()
        u.append(new_u)
        t_u += t2-t1
        t_d += t1-t0
    
    return u, t_d, t_u

# 10 runs

In [381]:
T = 0
T_u = 0
for t in range(1):
    G = G_origin.copy()
#     edge_list = getEdgeListUnique(G_origin, unodes_origin)
#     edge_vec = edgeUA(G_origin, unodes_origin)
    new_u, t_run, t_u = LR1Round(G, U, unodes_origin, STEP, CALCULATE_TIME, 'dk')
    u_of_10[t] = new_u
    T += t_run
    T_u += t_u
print("Average time: {:.3f}s".format(T/1))
print("Average measuring time: {:.3f}s".format(T_u/1))

Average time: 1764.153s
Average measuring time: 43.520s


In [383]:
with open('./imtermediate results/LR-ca-10-DK-1STEP.txt', 'wb') as f:
    pickle.dump(u_of_10, f)

In [382]:
for i in range(1, 10):
    u_of_10[i] = u_of_10[0]

# Delete 1% edges

In [352]:
# initialize
STEP = 25
CALCULATE_TIME = G_origin.ecount() // 100 // STEP
U, unodes_origin = uniqueness(G_origin, 'dk')
print(U)

x = list(range(STEP, (CALCULATE_TIME+1)*STEP, STEP))
u_of_10 = np.full((10, CALCULATE_TIME+1), U)

u_sum = 0
a_sum = 0
t_sum = 0

T = 0
t0 = time.time()
for t in range(1):
    G = G_origin.copy()
#     edge_list = getEdgeListUnique(G_origin, unodes_origin)
#     edge_vec = edgeUA(G_origin, unodes_origin)
    new_u, _, _ = LR1Round(G, U, unodes_origin, STEP, CALCULATE_TIME, 'dk')
    a_sum += G.transitivity_avglocal_undirected(mode="zero")
    t_sum += G.transitivity_undirected()
    u_sum += new_u[-1]
t1 = time.time()
print("Uniqueness: {:.3f}".format(u_sum/1))
print("ACC: {:.3f}".format(a_sum/1))
print("T: {:.3f}".format(t_sum/1))
print("Time: {:.3f}s".format(t1-t0))

0.13506295307134755
Uniqueness: 0.116
ACC: 0.520
T: 0.629
Time: 39.204s


In [ ]:
def raw_to_nauty(input_file, output_file, self_loop=False, debug=True,
                 comment="%", sep="\t", src_col=0, trgt_col=1, skiprows=0, int_type=True):
    f_in = open(input_file, "r")
    f_out = open(output_file, "w")
    
    edge_count = 0
    edge_count_undirected = 0
    node_dict = {}
    edge_list = {}
    node_count = 0
    self_edges = 0
    count = 0
    count_mod = 10
    skip_count = 0
    
    if debug:
        print("Reading " + str(len(f_in.readlines())) + " lines from file")
        print("Printing data of first 10...")
    f_in = open(input_file, "r")
    
    for ln in f_in:
        
        if ln.startswith(comment):
            continue
        if skiprows > skip_count:
            skip_count += 1
            continue
        
        split = ln.strip().split(sep)
        if int_type:
            src = int(split[src_col])
            trgt = int(split[trgt_col])
        else:
            src = split[src_col].strip()
            trgt = split[trgt_col].strip()
            
        
        # Print first couple of lines
        if debug and count < 10:
            print(split)
        if src == trgt:
            if not self_loop:
                continue
            self_edges += 1
        
        # Only add nodes with an edge
        if src not in node_dict:
            node_dict[src] = node_count
            node_count += 1
        if trgt not in node_dict:
            node_dict[trgt] = node_count
            node_count += 1
        
        src = node_dict[src]
        trgt = node_dict[trgt]
        
        try:
            edge_list[src].append(trgt)
        except:
            edge_list[src] = [trgt]
        try:
            edge_list[trgt].append(src)
        except:
            edge_list[trgt] = [src]

        count+= 1
        if debug and count % count_mod == 0:
            print("Processed edges: " + str(count))
            if count_mod < 100000000:
                count_mod *= 10
    
    # Count edges
    for key in edge_list:
        edgelist = np.unique(edge_list[key])
        edges = len(np.unique(edgelist))
        edge_count += edges * 0.5
        edge_count_undirected += edges
        if key in edgelist:
            edge_count += 0.5
                
    # To nauty format
    # Print number of nodes and edges
    f_out.write("!n=" + str(len(list(edge_list.keys()))) + "\n")
    f_out.write("!m=" + str(edge_count_undirected) + "\n")

    # Print edgelist
    for s in range(len(list(edge_list.keys()))):
        f_out.write(str(s) + ":")
        try:
            
            targets = np.unique(edge_list[s])
            for t in targets:
                f_out.write(" " + str(t))
                
            f_out.write(";\n")
        except:
            f_out.write(";\n")
    f_in.close()
    f_out.close()
    
    # Return stats
    stats = []
    stats.append(len(list(edge_list.keys())))
    stats.append(edge_count)
    stats.append(self_edges)
    return stats
    
raw_to_nauty("CA-GrQc.txt", "test", comment="#", self_loop=True)

In [353]:
with open('test', 'r') as f:
    lines = f.readlines()
cmd = "n 5242\ng\n".join(lines)

P = defaultdict(list)
cmd += "d " + str(1) + "\na\n"
cmd += ".\n"
r = subprocess.Popen(['./dkAnonymity/src/menu'], 
                            stdin=subprocess.PIPE, 
                            stdout=subprocess.PIPE, 
                            stderr=subprocess.PIPE, 
                            shell=True)
res = r.communicate(cmd.encode())[0].decode()
str_p = res.split("\n")
str_p = str_p[str_p.index("node, class_id")+1: str_p.index("End eq map")]

for i in str_p:
    p = i.split(", ")
    P[p[1]].append(int(p[0]))

u = 0
unique_nodes = []
for cl in P:
    if len(P[cl]) < 2:
        u += 1/5242
u

0.13506295307134755